In [2]:
!pip install pytorch-pretrained-bert

  Using cached pytorch_pretrained_bert-0.6.2-py3-none-any.whl (123 kB)
  Using cached boto3-1.24.7-py3-none-any.whl (132 kB)
  Using cached s3transfer-0.6.0-py3-none-any.whl (79 kB)
  Using cached jmespath-1.0.0-py3-none-any.whl (23 kB)
  Using cached botocore-1.27.7-py3-none-any.whl (8.9 MB)


In [3]:
!pip install transformers

In [4]:
from transformers.activations import gelu
from transformers import (BertTokenizer, BertConfig,
                          BertForSequenceClassification, BertPreTrainedModel,
                          apply_chunking_to_forward, set_seed,
                          )
from transformers.modeling_outputs import (BaseModelOutputWithPastAndCrossAttentions,
                                           BaseModelOutputWithPoolingAndCrossAttentions,
                                           SequenceClassifierOutput,
                                           )
                                           
from pytorch_pretrained_bert import BertModel

In [9]:
import torch

In [5]:
import pandas as pd
test = pd.read_csv(r'./onestopeng/test.csv')
#train = pd.read_csv(r'./onestopeng/train.csv')

In [17]:
train = pd.read_csv(r'./onestopeng/train.csv')

In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [6]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased')

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): BertLayerNorm()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): BertLayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (intermediate): BertIntermediate(
          (dense): Lin

In [7]:
input_texts = test.text.values.tolist()
labels = test.label.values.tolist()

input_ids = []
attention_masks = []

In [18]:
input_texts = train.text.values.tolist()
labels = train.label.values.tolist()

input_ids = []
attention_masks = []

In [19]:
for sent in input_texts:

    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 512,           # Pad & truncate all sentences.
                        #pad_to_max_length = True,
                        padding='max_length',
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                        truncation=True
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

    # Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0).long()
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

In [20]:
segments_tensors = torch.ones(510,512).long()

In [21]:
input_ids.shape, segments_tensors.shape

(torch.Size([510, 512]), torch.Size([510, 512]))

In [ ]:
embeddings = []

In [22]:

for i in range(0,510):
  with torch.no_grad():
    print(i,"/",510)
    encoded_layers, _ = model(input_ids[i,:].unsqueeze(0), segments_tensors[i,:].unsqueeze(0))
    embeddings.append(encoded_layers[11][0])

0 / 510
1 / 510
2 / 510
3 / 510
4 / 510
5 / 510
6 / 510
7 / 510
8 / 510
9 / 510
10 / 510
11 / 510
12 / 510
13 / 510
14 / 510
15 / 510
16 / 510
17 / 510
18 / 510
19 / 510
20 / 510
21 / 510
22 / 510
23 / 510
24 / 510
25 / 510
26 / 510
27 / 510
28 / 510
29 / 510
30 / 510
31 / 510
32 / 510
33 / 510
34 / 510
35 / 510
36 / 510
37 / 510
38 / 510
39 / 510
40 / 510
41 / 510
42 / 510
43 / 510
44 / 510
45 / 510
46 / 510
47 / 510
48 / 510
49 / 510
50 / 510
51 / 510
52 / 510
53 / 510
54 / 510
55 / 510
56 / 510
57 / 510
58 / 510
59 / 510
60 / 510
61 / 510
62 / 510
63 / 510
64 / 510
65 / 510
66 / 510
67 / 510
68 / 510
69 / 510
70 / 510
71 / 510
72 / 510
73 / 510
74 / 510
75 / 510
76 / 510
77 / 510
78 / 510
79 / 510
80 / 510
81 / 510
82 / 510
83 / 510
84 / 510
85 / 510
86 / 510
87 / 510
88 / 510
89 / 510
90 / 510
91 / 510
92 / 510
93 / 510
94 / 510
95 / 510
96 / 510
97 / 510
98 / 510
99 / 510
100 / 510
101 / 510
102 / 510
103 / 510
104 / 510
105 / 510
106 / 510
107 / 510
108 / 510
109 / 510
110 / 510


In [24]:
len(embeddings)

567

In [34]:
emb = torch.stack(embeddings, dim=0).numpy()

In [35]:
emb.shape

(567, 512, 768)

In [36]:
import pickle
pickle.dump(emb, open("Embedding_OSE.pickle", 'wb'))

In [37]:
data = pickle.load(open("Embedding_OSE.pickle", 'rb'))

In [38]:
data.shape

(567, 512, 768)